# Triangular arbitrage (Rust bindings)

Notebook d'exemple pour consommer les bindings Rust (hft_py) et analyser les opportunités triangulaires en temps réel.

Pré-requis : avoir build et installé la lib Rust Python (rust_python_bindings) ou rendre disponible le module hft_py dans l'environnement.

In [ ]:
# Imports et initialisation
import asyncio
import nest_asyncio
nest_asyncio.apply()
from datetime import datetime
import pandas as pd
from collections import defaultdict
try:
    from hft_py import PyAggregator
except Exception as e:
    print("Impossible d'importer hft_py : build/install la lib rust_python_bindings. Erreur:", e)


In [ ]:
# Créer l'aggregator et démarrer les connecteurs (exemple minimal)
agg = PyAggregator()
# démarre les connecteurs (squelettes). Chaque appel retourne un handle id (u64)
h_bin = agg.start_binance_ws(None)
h_kr = agg.start_kraken_ws(None)
h_cb = agg.start_coinbase_ws(None)
h_cg = agg.start_coingecko_poll(['bitcoin/usd','ethereum/usd'], 5000)
print('connectors handles:', h_bin, h_kr, h_cb, h_cg)


In [ ]:
# Subscription: stockage et affichage des ticks dans un DataFrame en temps réel
state = defaultdict(dict)

def on_tick_store(tick):
    key = f"{tick['exchange']}:{tick['pair']}"
    state[key] = tick
    ts = datetime.fromtimestamp(tick['ts']/1000.0)
    print(f"[{ts}] {tick['exchange']} {tick['pair']} bid={tick['bid']} ask={tick['ask']}")

async def run_sub_store():
    await agg.subscribe(on_tick_store)

asyncio.get_event_loop().create_task(run_sub_store())


## Recherche triangulaire (démo Python)
La détection qui suit est une implémentation pédagogique en Python. Pour production, implémente la logique en Rust et expose via hft_py pour la performance.

In [ ]:
def split_pair(p):
    if '-' in p: return p.split('-',1)
    if '/' in p: return p.split('/',1)
    if len(p) > 6:
        return [p[:-3], p[-3:]]
    return [p, '']

def find_triangular_opportunities(snapshot_state, threshold=0.001):
    """Recherche brute de cycles A->B->C sur snapshots (niveau 1).
    Retourne liste d'opportunités: {'path':(k1,k2,k3),'profit':float}
    """
    opps = []
    keys = list(snapshot_state.keys())
    n = len(keys)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i==j or j==k or i==k: continue
                a = snapshot_state.get(keys[i])
                b = snapshot_state.get(keys[j])
                c = snapshot_state.get(keys[k])
                if not a or not b or not c: continue
                try:
                    # naive round-trip: buy at A.ask, sell B at B.bid, sell C at C.bid
                    ratio = (1.0 / a['ask']) * b['bid'] * c['bid']
                    profit = ratio - 1.0
                    if profit > threshold:
                        opps.append({'path':(keys[i],keys[j],keys[k]), 'profit': profit})
                except Exception:
                    continue
    return opps

import time
def periodic_search(interval=5, rounds=12):
    for _ in range(rounds):
        opps = find_triangular_opportunities(state, threshold=0.001)
        if opps:
            print('Opportunités trouvées:')
            for o in opps:
                print(o)
        else:
            print('Aucune opportunité détectée')
        time.sleep(interval)

asyncio.get_event_loop().run_in_executor(None, periodic_search, 5, 12)
